# Exercício Aula 07 - Gerando Redes c/ Modelo BA e Configuration

Author: Gabriel Van Loon

Prof.:  Francisco Aparecido Rodrigues

Universidade de São Paulo, São Carlos, Brasil.


## Definindo as Funções e Bibliotecas utilizadas

In [43]:
import collections
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from scipy.stats import pearsonr, spearmanr

from networkx.algorithms import community, centrality
from networkx.algorithms.community import greedy_modularity_communities
from community import community_louvain

In [5]:
def degree_distribution(GER):
    vk = dict(GER.degree())
    vk = list(vk.values()) # we get only the degree values
    maxk = np.max(vk)
    mink = np.min(min)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

def momment_of_degree_distribution(G,m):
    k,Pk = degree_distribution(G)
    M = sum((k**m)*Pk)
    return M

def shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H


def modularity(G, c):
    """
    Calcula a modularidade da partição realizada na rede G nas c
    comunidades. Outra opção é utilizar a função do networkx
    networkx.algorithms.community.quality.modularity(G,c)
    """
    A = nx.adjacency_matrix(G)
    N = len(G)
    M = G.number_of_edges()
    Q = 0
    for i in np.arange(0,N):
        ki = len(list(G.neighbors(i)))
        for j in np.arange(0,N):
            if(c[i]==c[j]):
                kj = len(list(G.neighbors(j)))
                Q = Q + A[i,j]-(ki*kj)/(2*M)
    Q = Q/(2*M)
    return Q

## 1) Calcule a média do coeficiente de aglomeração e segundo momento do grau para uma rede BA com grau medio igual a 10 e n=1000

In [152]:
N = 1000
av_degree = 8
m = int(av_degree/2)
G = nx.barabasi_albert_graph(N, m)

In [153]:
avg_deg = 2*G.number_of_edges()/N
moment  = momment_of_degree_distribution(G,2)
clustering = nx.average_clustering(G)

print("Average clustering:", clustering)
print("Second Moment", moment)
print("Average Degree:", avg_deg)

Average clustering: 0.03434954127818049
Second Moment 135.94400000000002
Average Degree: 7.968


## 2) Considere uma rede aleatória (Erdos-Renyi) e uma BA com N=1000 vértices e grau médio 10. Qual o valor da entropia de Shannon da distrbuição do grau para essas redes?

In [154]:
N, p = (1000, 0.01) # <k> = 10
GER = nx.gnp_random_graph(N, p, seed=None, directed=False)

N = 1000
av_degree = 10
m = int(av_degree/2)
GBA = nx.barabasi_albert_graph(N, m)

In [155]:
print("Shannon Entropy ER = ", "%3.4f"%shannon_entropy(GER))
print("Shannon Entropy BA = ", "%3.4f"%shannon_entropy(GBA))

Shannon Entropy ER =  3.6741
Shannon Entropy BA =  3.5949


## 3) Considere o modelo de Barabarsi-Albert om N=1000 e grau médio igual a 10. Calcule o coeficiente de correlação de Pearson (rho) entre o grau e a medida eigenvector centrality. O que esse valor indica?

In [52]:
eigen_centrality    = dict(nx.eigenvector_centrality(GBA))
degree_centrality = dict(nx.degree_centrality(GBA))

pearson_corr, _ = pearsonr(list(eigen_centrality.values()), list(degree_centrality.values()))

print("Pearson Correlation:", "%3.4f"%pearson_corr)]
# Re: 0,9 indica que os hubs são os mais visitados em uma caminha aleatória.

Pearson Correlation: 0.9400


## 4) Calcule a correlação entre a medida de betweeness centrality e o grau para uma rede BA. Considerada N=500 e grau médio 10

In [156]:
N = 500
av_degree = 10
m = int(av_degree/2)
GBA = nx.barabasi_albert_graph(N, m)

In [157]:
bet_centrality    = dict(nx.betweenness_centrality(GBA))
degree_centrality = dict(nx.degree_centrality(GBA))

pearson_corr, _ = pearsonr(list(bet_centrality.values()), list(degree_centrality.values()))

print("Pearson Correlation:", "%3.4f"%pearson_corr)

Pearson Correlation: 0.9674


## 5) Calcule o segundo momento do grau para o modelo de configuração com a=3 (coef da lei de potência (Zipf)). Considere N=500 e o valor mais próximo, pois os valores podem variar de uma simulação para outra.

In [165]:
# Primeiro precisamos montar a distribuição que queremos modelar 
N = 500
a = 3.0
seq = np.random.zipf(a, N) # Zipf distribution

# Error preventing
if(sum(seq)%2 != 0): # the sum of stubs have to be even
    pos = np.random.randint(0, len(seq))
    seq[pos] = seq[pos]+ 1

# Passamos a configuração para o modelo :P
GCM=nx.configuration_model(seq)

# Pegando o maior componente
Gcc = sorted(nx.connected_components(GCM), key=len, reverse=True)
GCM = GCM.subgraph(Gcc[0])

In [166]:
print("Second Moment", momment_of_degree_distribution(GCM,2))

Second Moment 14.916030534351144
